<a href="https://colab.research.google.com/github/escaduto/Building-Extraction-with-Deep-Learning/blob/main/GEE_MultiDir_Hillshade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting Up

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive

drive.mount(ROOT)   

Mounted at /content/drive


In [ ]:
import os
rootPath = '/content/drive/My Drive/hillshade product'
os.chdir(rootPath)

In [ ]:
!pip install geopandas
!pip install rasterio

In [ ]:
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
from pathlib import Path
import rasterio
import os

## Connect to GEE

In [ ]:
# initialize and connect to GEE 
from google.colab import auth
auth.authenticate_user()
# !earthengine authenticate
import ee 
ee.Initialize()

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# GET US Data Products (10m)

## USGS 3DEP National Map Seamless 10m (USA)

In [ ]:
USGS_3DEP_10m = ee.Image('USGS/3DEP/10m').multiply(4)
USGS_3DEP_10m_ELEV = USGS_3DEP_10m.select('elevation')
USGS_3DEP_10m_SLOPE = ee.Terrain.slope(USGS_3DEP_10m_ELEV)

## USGS NED 10m (USA)

In [ ]:
USGS_NED_10m = ee.Image('USGS/NED')
USGS_NED_10m_ELEV = USGS_NED_10m.select('elevation')
USGS_NED_10m_SLOPE = ee.Terrain.slope(USGS_NED_10m_ELEV)

# GET Global Data Products (30m)

## SRTM 30m (GLOBAL)

In [ ]:
SRTM = ee.Image('USGS/SRTMGL1_003').multiply(4)

## ALOS Digital Surface Model 30m (GLOBAL)

In [ ]:
# /* ALOS DSM: Global 30m */
ALOS = ee.Image('JAXA/ALOS/AW3D30/V2_2').multiply(4)

## ETOPO1 Bathymetry 30m (GLOBAL)

In [ ]:
# /* ETOPO1: Global 1 Arc-Minute Elevation */
NOAA = ee.Image('NOAA/NGDC/ETOPO1')

## Water Layer

In [ ]:
# /* JRC Global Surface Water Mapping Layers, v1.2 */
GSWM = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')

# Apply Hillshade

In [ ]:
def multiDir_Hillshade(ALOS):
  N = ee.Terrain.hillshade(ALOS,0,36).multiply(0);
  NE = ee.Terrain.hillshade(ALOS,45,44).multiply(0);
  E = ee.Terrain.hillshade(ALOS,90,56).multiply(0);
  SE = ee.Terrain.hillshade(ALOS,135,68).multiply(0);
  S = ee.Terrain.hillshade(ALOS,180,80).multiply(0.1);
  SW = ee.Terrain.hillshade(ALOS,225,68).multiply(0.2);
  W = ee.Terrain.hillshade(ALOS,270,56).multiply(0.2);
  NW = ee.Terrain.hillshade(ALOS,315,44).multiply(0.5);
  MULTI = N.add(NE).add(E).add(SE).add(S).add(SW).add(W).add(NW).visualize(**{'min':0,'max':255,'palette':[
        '#000000',
        '#ffffff'
        ],
    }).resample('bicubic').updateMask(0.5)
  SLOPE = ee.Terrain.slope(ALOS).visualize(**{'min':0,'max':180,'palette':['#ffffff','#000000']}).resample('bicubic').updateMask(1)
  SHADED_RELIEF = ee.ImageCollection([SLOPE,MULTI]).mosaic().reduce(ee.Reducer.median()).updateMask(1)
  return SHADED_RELIEF

### Define Traditional Hillshade Weights

In [ ]:
# /* TERRAIN */
# /* Traditional Hillshade (input,azimuth,altitude).multiply(weight) */
N = ee.Terrain.hillshade(ALOS,0,36).multiply(0);
NE = ee.Terrain.hillshade(ALOS,45,44).multiply(0);
E = ee.Terrain.hillshade(ALOS,90,56).multiply(0);
SE = ee.Terrain.hillshade(ALOS,135,68).multiply(0);
S = ee.Terrain.hillshade(ALOS,180,80).multiply(0.1);
SW = ee.Terrain.hillshade(ALOS,225,68).multiply(0.2);
W = ee.Terrain.hillshade(ALOS,270,56).multiply(0.2);
NW = ee.Terrain.hillshade(ALOS,315,44).multiply(0.5);

### Create Multidirectional Hillshade Layer

In [ ]:
# /* Multidirectional Hillshade */
MULTI = N.add(NE).add(E).add(SE).add(S).add(SW).add(W).add(NW).visualize(**{'min':0,'max':255,'palette':[
        '#000000',
        '#ffffff'
        ],
    }).resample('bicubic').updateMask(0.5)

# Apply Slope

In [ ]:
# /* Slope */
SLOPE = ee.Terrain.slope(ALOS).visualize(**{'min':0,'max':180,'palette':['#ffffff','#000000']}).resample('bicubic').updateMask(1)

# Shaded Relief 

Combination of Multidirectional hillshade and slope

In [ ]:
SHADED_RELIEF = ee.ImageCollection([SLOPE,MULTI]).mosaic().reduce(ee.Reducer.median()).updateMask(1)

# Other Layers

## Hypsometric Tinting

In [ ]:
# /* Elevation */
ELEVATION = ALOS.visualize(**{'bands':['AVE_DSM'],'min':0,'max':12500,
        'palette':[
            '#386641',
            '#6a994e',
            '#a7c957',
            '#fdf7d6',
            '#ffffff'
            ]
        }).resample('bicubic').updateMask(0.3)

## Add Surface Water

In [ ]:
SURFACE_WATER = GSWM.visualize(**{'bands':['occurrence'],
        'min':0,
        'max':100,
        'palette':[
            '#B9E9E7'
            ]
        }).resample('bicubic').updateMask(1)

## Add Ocean Water

In [ ]:
OCEAN = ALOS.updateMask(ALOS.lte(0)).visualize(**{
        'bands':['AVE_DSM'],
        'min':0,
        'max':0,
        'palette':[
            'B9E9E7'
            ]
        }).resample('bicubic');

## Add Bathymetry 

In [ ]:
BATHYMETRY = NOAA.updateMask(NOAA.lte(-10)).visualize(**{
        'bands':['bedrock'],
        'min':-5000,
        'max':0,
        'palette':[
            '#8ECCCB',
            '#ABE0DF',
            'B9E9E7'
            ]
        }).resample('bicubic');

# Visualize

In [ ]:
USGS_DEM_10m = multiDir_Hillshade(USGS_3DEP_10m)
ALOS_DSM_30m = multiDir_Hillshade(ALOS)
SRTM_DEM_30m = multiDir_Hillshade(SRTM)

In [92]:
Map = emap.Map(center=[37.7799, -122.4509], zoom=13)
Map.add_basemap('TERRAIN')

Map.addLayer(ALOS_DSM_30m, {'min':0,'max':255,'gamma':1}, 'ALOS_DSM_30m') 
Map.addLayer(SRTM_DEM_30m, {'min':0,'max':255,'gamma':1}, 'SRTM_DEM_30m') 
Map.addLayer(USGS_DEM_10m, {'min':0,'max':255,'gamma':1}, 'USGS_3DEP_10m')
 
# Map.addLayer(ELEVATION,{},'TINTED')
# Map.addLayer(SURFACE_WATER,{},'Surface Water')
# Map.addLayer(OCEAN,{},'OCEAN')
# Map.addLayer(BATHYMETRY,{},'BATHYMETRY', 'false')
Map.addLayerControl()
Map

# Export to Drive Folder

In [ ]:
rootPath = '/content/drive/My Drive/hillshade product/results'
os.chdir(rootPath)

In [ ]:
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

CA_geom = ee.FeatureCollection('users/escaduto/CA_FireTrends/CA_Extent').geometry()
US_geom = ee.FeatureCollection('projects/ee-globaldem/assets/USA_Boundary').geometry()

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': USGS_DEM_10m,
    'description': 'USGS_DEM_CA_10m',
    'folder':'USGS_DEM_CA_10m',
    'scale': 10,
    'region': CA_geom,
    'maxPixels': 1e13
})
task.start()

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': SHADED_RELIEF,
    'description': 'shaderelief_USA_30m',
    'folder':'Shaded_Relief_USA_30m',
    'scale': 30,
    'region': US_geom,
    'maxPixels': 1e13
})
task.start()

In [ ]:
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

# Export to Cloud Bucket

In [ ]:
outputBucket = 'global_hillshade' #Change for your Cloud Storage bucket

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toCloudStorage(**{
    'image': SHADED_RELIEF,
    'description': 'shaderelief_USA_30m',
    'scale': 30,
    'region': US_geom,
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
    'bucket': outputBucket,
    'formatOptions': {'cloudOptimized': True}
})
task.start()

In [ ]:
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(30)
else: 
  print('Completed task (id: {})!!'.format(task.id))

# Retrieve tif files in folder 

In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

In [ ]:
dirpath = "/content/drive/My Drive/hillshade product/results/Shaded_Relief_v1"

In [ ]:
os.chdir("/content/drive/My Drive/hillshade product/results/Shaded_Relief_v1")
dem_fps = os.listdir()

In [ ]:
dem_fps

['shaderelief_export-0000000000-0000000000.tif',
 'shaderelief_export-0000000000-0000023296.tif',
 'shaderelief_export-0000023296-0000000000.tif',
 'shaderelief_export-0000023296-0000023296.tif']

# Create Continuous tif & Reduce size

In [ ]:
src_files_to_mosaic = []

In [ ]:
for fp in dem_fps:
  src = rasterio.open(fp)
  src_files_to_mosaic.append(src)

src_files_to_mosaic

[<open DatasetReader name='shaderelief_export-0000000000-0000000000.tif' mode='r'>,
 <open DatasetReader name='shaderelief_export-0000000000-0000023296.tif' mode='r'>,
 <open DatasetReader name='shaderelief_export-0000023296-0000000000.tif' mode='r'>,
 <open DatasetReader name='shaderelief_export-0000023296-0000023296.tif' mode='r'>]

In [ ]:
mosaic, out_trans = merge(src_files_to_mosaic)

In [ ]:
show(mosaic, cmap='terrain')

In [ ]:
# Copy the metadata
out_meta = src.meta.copy()

# Update the metadata
out_meta.update({"driver": "GTiff","height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans})


In [ ]:
out_fp = 'Shaded_Relief_CA_30m.tif'
with rasterio.open(out_fp, "w", **out_meta) as dest:
  dest.write(mosaic)